# Imports

In [27]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# from ydata_profiling import ProfileReport
# from ydata_quality import DataQuality
# from ydata_quality.erroneous_data import ErroneousDataIdentifier
# from ydata_quality.duplicates import DuplicateChecker
# from ydata_quality.missings import MissingsProfiler
import great_expectations as gx

# EDA

## Madrid

### Loading the data

In [2]:
df = pd.read_csv('houses_Madrid.csv')
# df.head()

### Dropping empty columns

In [3]:
empty_cols = [col for col in df.columns if df[col].isnull().all()]
df = df.drop(empty_cols, axis=1)
# df.info()

### Handling boolean columns

In [4]:
# Bool columns
columns_bool = [
    'is_exact_address_hidden',
    'is_renewal_needed',
    'has_parking'
]

# Columns with TRUE/FALSE/blank
columns_true_false_blank = [
    'is_floor_under',
    'is_new_development',
    'has_central_heating',
    'has_individual_heating',
    'has_lift',
    'is_parking_included_in_price',
    'is_orientation_north',
    'is_orientation_west',
    'is_orientation_south',
    'is_orientation_east'
]

# Columns with TRUE/blank
columns_true_blank = [
    'has_ac',
    'has_fitted_wardrobes',
    'is_exterior',
    'has_garden',
    'has_pool',
    'has_terrace',
    'has_balcony',
    'has_storage_room',
    'is_accessible',
    'has_green_zones'
]

In [5]:
# Mapping function for boolean columns
def map_bool(val):
    if val == True:
        return 1
    else:
        return 0

# Mapping function for TRUE/FALSE/blank columns
def map_true_false_blank(val):
    if np.isnan(val):
        return np.nan
    elif val == True:
        return 1
    else:
        return 0

# Mapping function for TRUE/blank columns
def map_true_blank(val):
    if val == True:
        return 1
    else:
        return 0

In [6]:
# print the head of the columns_true_false_blank
# df[columns_true_false_blank].head()

In [7]:
# Apply mappings
for col in columns_bool:
    df[col] = df[col].map(map_bool)

for col in columns_true_false_blank:
    df[col] = df[col].map(map_true_false_blank)

for col in columns_true_blank:
    df[col] = df[col].map(map_true_blank)

In [8]:
# df.info()

### Handling categorical columns

In [9]:
# df['neighborhood_id'].value_counts()

In [10]:
# extract the neighborhood name, and the district name from the neighborhood_id
chop = df['neighborhood_id'].str.extract(r'Neighborhood (\d+): (.*?) \(.*\) - District (\d+): (.*)')
df['neighborhood'] = chop[1]+ ' ' + chop[0]
df['district'] = chop[3] + ' ' + chop[2]
df.drop(axis=1, columns=['neighborhood_id'], inplace=True)
# df[['neighborhood', 'district']].head()

In [11]:
# Categorical columns
columns_categorical = [
    'neighborhood',
    'district',
    'house_type_id',
    'energy_certificate',
]

columns_categorical += columns_bool
columns_categorical += columns_true_false_blank
columns_categorical += columns_true_blank

In [12]:
# Convert to categorical
df[columns_categorical] = df[columns_categorical].astype('category')
# df.info()

### Creating column lists

In [13]:
# create a list of column names that are not numerical
non_numerical_columns = df.select_dtypes(include=['object']).columns.to_list()

# create a list of column names that are int64 or float64 and exclude the 'id' column
numerical_columns = df.select_dtypes(include=['int64', 'float64']).columns.to_list()
numerical_columns.remove('id')

# create a list of column names that are categorical
categorical_columns = df.select_dtypes(include=['category']).columns.to_list()

In [14]:
df[numerical_columns].describe()

,sq_mt_built,sq_mt_useful,n_rooms,n_bathrooms,n_floors,sq_mt_allotment,rent_price,buy_price,buy_price_by_area,built_year,parking_price
count,21616.000000,8228.000000,21742.000000,21726.000000,1437.000000,1432.000000,2.174200e+04,2.174200e+04,21742.000000,10000.000000,7719.000000
mean,146.920892,103.458192,3.005749,2.091687,3.128740,241.692737,-5.917031e+04,6.537356e+05,4020.523871,1970.046400,2658.000518
std,134.181865,88.259192,1.510497,1.406992,0.907713,247.484853,9.171162e+05,7.820821e+05,1908.418774,69.386705,13360.966258
min,13.000000,1.000000,0.000000,1.000000,1.000000,1.000000,-3.459028e+07,3.600000e+04,447.000000,1723.000000,0.000000
25%,70.000000,59.000000,2.000000,1.000000,2.000000,2.000000,7.250000e+02,1.980000e+05,2551.000000,1957.000000,0.000000
50%,100.000000,79.000000,3.000000,2.000000,3.000000,232.000000,1.116000e+03,3.750000e+05,3720.000000,1970.000000,0.000000
75%,162.000000,113.000000,4.000000,2.000000,4.000000,354.000000,1.687000e+03,7.636000e+05,5000.000000,1994.000000,0.000000
max,999.000000,998.000000,24.000000,16.000000,7.000000,997.000000,2.517000e+03,8.800000e+06,18889.000000,8170.000000,600000.000000


In [15]:
df_num = df[numerical_columns].copy()
df_num_cat = df[numerical_columns + categorical_columns].copy()

## Energy Consumption

### Loading the data

In [28]:
# Loading meta data for each of the houses
meta = pd.read_csv('energy_consumption/residential_meta.csv')
meta.rename(columns={'House':'HouseID'}, inplace=True)
meta.drop(columns=['RUs', 'Cover'], inplace=True)

In [29]:
meta['FirstReading'] = pd.to_datetime(meta['FirstReading'])
meta['LastReading'] = pd.to_datetime(meta['LastReading'])
meta['Duration'] = meta['LastReading'] - meta['FirstReading']
meta['Duration'] = meta['Duration'].dt.days

In [31]:
# Read every csv file in the energy_consumption/Residential folder and before concatenating them, map the meta data to each of the houses using the House ID
# and the name of the Residential csv file which is in the format of Residential_<HouseID>.csv
dfs = []
for file in os.listdir('energy_consumption/Residential'):
    edf = pd.read_csv('energy_consumption/Residential/' + file)
    edf['HouseID'] = file.split('_')[1].split('.')[0]
    edf['HouseID'] = edf['HouseID'].astype('int64')
    edf = edf.merge(meta, on='HouseID')
    dfs.append(edf)

edf = pd.concat(dfs)
edf.rename(columns={'energy_kWh': 'kWh'}, inplace=True)
edf.head()

,date,hour,kWh,HouseID,FirstReading,LastReading,HouseType,Facing,Region,FAGF,...,FPE,IFRHG,NAC,FAC,PAC,BHE,IFRHE,WRHIR,GEOTH,Duration
0,2012-06-01,1,1.011,1,2012-06-01,2015-10-03,bungalow,South,YVR,1,...,0,0,0,0,0,0,0,0,0,1219
1,2012-06-01,2,0.451,1,2012-06-01,2015-10-03,bungalow,South,YVR,1,...,0,0,0,0,0,0,0,0,0,1219
2,2012-06-01,3,0.505,1,2012-06-01,2015-10-03,bungalow,South,YVR,1,...,0,0,0,0,0,0,0,0,0,1219
3,2012-06-01,4,0.441,1,2012-06-01,2015-10-03,bungalow,South,YVR,1,...,0,0,0,0,0,0,0,0,0,1219
4,2012-06-01,5,0.468,1,2012-06-01,2015-10-03,bungalow,South,YVR,1,...,0,0,0,0,0,0,0,0,0,1219


### Handling categorical columns

In [33]:
# Categorical columns
e_categorical_columns = [
    'HouseID',
    'HouseType',
    'Facing',
    'Region',
]

In [34]:
# Convert to categorical
edf[e_categorical_columns] = edf[e_categorical_columns].astype('category')

### Creating column lists

In [35]:
e_numerical_columns = ['hour', 'kWh', 'Duration']

In [36]:
e_df_num = edf[e_numerical_columns].copy()
e_df_num_cat = edf[e_numerical_columns + e_categorical_columns].copy()

In [37]:
e_df_num_cat.head()

,hour,kWh,Duration,HouseID,HouseType,Facing,Region
0,1,1.011,1219,1,bungalow,South,YVR
1,2,0.451,1219,1,bungalow,South,YVR
2,3,0.505,1219,1,bungalow,South,YVR
3,4,0.441,1219,1,bungalow,South,YVR
4,5,0.468,1219,1,bungalow,South,YVR


## Data Quality

### Great Expectations

In [16]:
path_to_folder = 'C:\\Users\\bened\\Documents\\Fairness in AI\\'

In [17]:
context = gx.data_context.FileDataContext(project_root_dir=path_to_folder)
# print(context)

In [18]:
# Initialize the data source and data asset
data_source = context.sources.add_or_update_pandas(name='houses_madrid')
ds_name = 'houses_madrid'
data_asset = data_source.add_dataframe_asset(name=ds_name)

In [19]:
data_asset = context.get_datasource(ds_name).get_asset(ds_name)
batch_request = data_asset.build_batch_request(dataframe=df_num_cat)

In [20]:
context.add_or_update_expectation_suite('default')

{
  "expectation_suite_name": "default",
  "ge_cloud_id": null,
  "expectations": [],
  "data_asset_type": null,
  "meta": {
    "great_expectations_version": "0.18.12"
  }
}

In [21]:
validator = context.get_validator(batch_request=batch_request, expectation_suite_name='default')
# validator.head()

#### Generic expectations

In [44]:
def test_data_quality(df, dq_name, categorical_columns=[], numeric_columns=[], export=False):
    # Initialize the data source and data asset for the given dataframe
    # and create a batch request for the validator, and get the validator
    data_asset = context.get_datasource(ds_name).get_asset(ds_name)
    batch_request = data_asset.build_batch_request(dataframe=df)
    context.add_or_update_expectation_suite(dq_name)
    validator = context.get_validator(batch_request=batch_request, expectation_suite_name=dq_name)

    # Initialize success rate counter
    success_count = 0
    total_expectations = 0

    # Test expectations for categorical columns
    for column in categorical_columns:
        success_count += 1 if validator.expect_column_values_to_not_be_null(column, mostly=0.85).success else 0
        total_expectations += 1
        
    # Test expectations for numeric columns
    for column in numeric_columns:
        success_count += 1 if validator.expect_column_values_to_not_be_null(column, mostly=0.85).success else 0
        success_count += 1 if validator.expect_column_stdev_to_be_between(column, min_value=0.5, max_value=3).success else 0
        total_expectations += 2

    validator.save_expectation_suite(discard_failed_expectations=False)

    if export:
        checkpoint = context.add_or_update_checkpoint(
            name=f"{dq_name}_checkpoint",
            validator=validator,
        )
        checkpoint.run()

    success_rate = success_count / total_expectations
    return success_rate

In [ ]:
# was inside the test_data_quality function
    # checkpoint = context.add_or_update_checkpoint(
    #     name=f"{dq_name}_checkpoint",
    #     validator=validator,
    # )

    # checkpoint_result = checkpoint.run()

    # # Calculate success rate
    # success_count = checkpoint_result.list_validation_results()[0]['statistics']['successful_expectations']

In [26]:
test_data_quality(df_num_cat, 'data_quality_test_2', categorical_columns, numerical_columns)

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

...

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

0.5263157894736842

In [45]:
test_data_quality(e_df_num_cat, 'data_quality_test_3', e_categorical_columns, e_numerical_columns, export=True)

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

...

Calculating Metrics:   0%|          | 0/4 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/4 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/41 [00:00<?, ?it/s]

0.8

### Tests

In [29]:
# validator.expect_column_values_to_not_be_null(column='buy_price')
# validator.expect_column_min_to_be_between(column='rent_price', min_value=0, max_value=10000000)
validator.expect_column_mean_to_be_between(column='rent_price', min_value=0, max_value=1000000)
# come up with 5 basic expectations for the dataset
validator.expect_column_values_to_be_in_set(column='house_type_id', value_set=['House', 'Flat', 'Studio'])
# validator.expect_column_values_to_be_unique(column='id')

validator.save_expectation_suite(discard_failed_expectations=False)

Calculating Metrics:   0%|          | 0/4 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

In [30]:
checkpoint = context.add_or_update_checkpoint(
    name="first_checkpoint",
    validator=validator,
)

In [31]:
checkpoint_result = checkpoint.run()
# context.view_validation_result(checkpoint_result)
# context.build_data_docs()

Calculating Metrics:   0%|          | 0/11 [00:00<?, ?it/s]

In [29]:
checkpoint_result.get_statistics()

{'data_asset_count': 1,
 'validation_result_count': 1,
 'successful_validation_count': 0,
 'unsuccessful_validation_count': 1,
 'successful_validation_percent': 0.0,
 'validation_statistics': {ValidationResultIdentifier::default/__none__/20240410T143612.448261Z/houses_madrid-houses_madrid: {'evaluated_expectations': 2,
   'successful_expectations': 0,
   'unsuccessful_expectations': 2,
   'success_percent': 0.0}}}

In [49]:
checkpoint_result.list_validation_results()

[{
   "success": false,
   "results": [
     {
       "success": false,
       "expectation_config": {
         "expectation_type": "expect_column_mean_to_be_between",
         "kwargs": {
           "column": "rent_price",
           "max_value": 1000000,
           "min_value": 0,
           "batch_id": "houses_madrid-houses_madrid"
         },
         "meta": {}
       },
       "result": {
         "observed_value": -59170.30792935332
       },
       "meta": {},
       "exception_info": {
         "raised_exception": false,
         "exception_traceback": null,
         "exception_message": null
       }
     },
     {
       "success": false,
       "expectation_config": {
         "expectation_type": "expect_column_values_to_be_in_set",
         "kwargs": {
           "column": "house_type_id",
           "value_set": [
             "House",
             "Flat",
             "Studio"
           ],
           "batch_id": "houses_madrid-houses_madrid"
         },
         "meta":

In [52]:
# get the number of successful expectations from the checkpoint result
success_count = checkpoint_result.list_validation_results()[0]['statistics']['successful_expectations']
print(success_count)

0


### YData Quality

In [44]:
dc = DuplicateChecker(df=df, entities=['neighborhood', 'District'])
# dc = DuplicateChecker(df)
results = dc.evaluate()

INFO | No exact duplicates were found.
INFO | No duplicate columns were found.
No warnings found.


In [40]:
edi = ErroneousDataIdentifier(df=df, ed_extensions=['NaN', np.nan, None])
results = edi.evaluate()

INFO | No predefined ED values from  the set {'', nan, 'unknown', None, '(blank)', 'n/a', '?', 'na', 'unk', 'nan'} were found in the dataset.
No warnings found.


In [41]:
flatlines_out = edi.flatlines(th=5)
print(flatlines_out)

None


In [46]:
mp = MissingsProfiler(df)
results = mp.evaluate()

Warnings:
	TOTAL: 3 warning(s)
	Priority 2: 1 warning(s)
	Priority 3: 2 warning(s)


Priority 2 - usage allowed, limited human intelligibility:
	* [MISSINGS - MISSINGNESS PREDICTION] Found 8 features with prediction performance                         of missingness above threshold (0.8).
Priority 3 - minor impact, aesthetic:
	* [MISSINGS - HIGH MISSING CORRELATIONS] Found 21 feature pairs with correlation of missing values higher than defined threshold (0.5).
	* [MISSINGS - HIGH MISSINGS] Found 20 columns with more than 20.0% of missing values.


In [11]:
dq = DataQuality(df)

In [12]:
result = dq.evaluate()

c:\Users\bened\AppData\Local\Programs\Python\Python38\lib\site-packages\ydata_quality\utils\correlations.py:73: RuntimeWarning: invalid value encountered in double_scalars
  return np.sqrt(phi_sq_hat / np.min([k_hat - 1, r_hat - 1]))  # Note: this is strictly positive
c:\Users\bened\AppData\Local\Programs\Python\Python38\lib\site-packages\ydata_quality\utils\correlations.py:73: RuntimeWarning: invalid value encountered in double_scalars
  return np.sqrt(phi_sq_hat / np.min([k_hat - 1, r_hat - 1]))  # Note: this is strictly positive
c:\Users\bened\AppData\Local\Programs\Python\Python38\lib\site-packages\ydata_quality\utils\correlations.py:73: RuntimeWarning: invalid value encountered in double_scalars
  return np.sqrt(phi_sq_hat / np.min([k_hat - 1, r_hat - 1]))  # Note: this is strictly positive
c:\Users\bened\AppData\Local\Programs\Python\Python38\lib\site-packages\ydata_quality\utils\correlations.py:73: RuntimeWarning: invalid value encountered in double_scalars
  return np.sqrt(phi_

LinAlgError: SVD did not converge